In [1]:
!pip install nltk

     |████████████████████████████████| 1.5 MB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 76 kB 779 kB/s  eta 0:00:01
     |████████████████████████████████| 764 kB 851 kB/s eta 0:00:01


In [2]:
import nltk

In [3]:
from nltk.tokenize import word_tokenize, sent_tokenize

In [4]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [6]:
import pandas as pd

data = pd.read_csv("/home/ibab/Desktop/ml_proj/train.csv/train.csv")
data.head()

,id,text,harsh,extremely_harsh,vulgar,threatening,disrespect,targeted_hate
0,52e0f91a5d7b74552c55,New Main Picture \n\nHow about this for the ma...,0,0,0,0,0,0
1,e2c8e370a8e53ba26bae,Think of them like population charts. Just bec...,0,0,0,0,0,0
2,03c807f61149a13c8404,This page seems a little misleading. The reaso...,0,0,0,0,0,0
3,fc63a1ba3372899db19f,"""\n\nActually, accounts are never deleted. """,0,0,0,0,0,0
4,0c2bfd9cde8974d9915f,"""\n\nYeah yeah, OK. So did I. Still, what I ...",0,0,0,0,0,0


In [10]:
list(data)

['id',
 'text',
 'harsh',
 'extremely_harsh',
 'vulgar',
 'threatening',
 'disrespect',
 'targeted_hate']

In [9]:
for i, j in data.head().iterrows():
    print(j)

id                                              52e0f91a5d7b74552c55
text               New Main Picture \n\nHow about this for the ma...
harsh                                                              0
extremely_harsh                                                    0
vulgar                                                             0
threatening                                                        0
disrespect                                                         0
targeted_hate                                                      0
Name: 0, dtype: object
id                                              e2c8e370a8e53ba26bae
text               Think of them like population charts. Just bec...
harsh                                                              0
extremely_harsh                                                    0
vulgar                                                             0
threatening                                                        0
disrespect 

In [10]:
data.shape

(127656, 8)

In [19]:
data.loc[122939,'text']

'Why you Little...\nI hope you get banned from Wikipedia someday! And you will! We will meet again you stupid Singaporan fool! But for now, I shall take my break from you, so long FOOL! HAHAHAHAHAHAHAHAHAHAHAHAHA! (Disapears into a puff of smoke).'

In [21]:
!pip install tensorflow

     |████████████████████████████████| 458.4 MB 19 kB/s s eta 0:00:01|████▍                           | 62.1 MB 12.3 MB/s eta 0:00:33
     |████████████████████████████████| 1.1 MB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 14.9 MB 89 kB/s eta 0:00:0151
  Using cached wheel-0.37.0-py2.py3-none-any.whl (35 kB)
     |████████████████████████████████| 463 kB 10.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 3.9 MB 418 kB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 89 kB/s eta 0:00:0101
     |████████████████████████████████| 57 kB 634 kB/s eta 0:00:01
     |████████████████████████████████| 132 kB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 607 kB/s eta 0:00:01
     |████████████████████████████████| 42 kB 228 kB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 78 kB/s s eta 0:00:01
     |████████████████████████████████| 155 kB 6.4 MB/s 

Preprocessing using tensorflow

In [38]:
import tensorflow as tf
from tensorflow import keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [39]:
#split input into training and testing

sentences = data['text']
labels = data[['harsh','extremely_harsh','vulgar','threatening','disrespect','targeted_hate']]
training_sentences,testing_sentences,training_lab,testing_lab = train_test_split(sentences,labels,test_size=.2)

In [40]:
vocab_size = 100000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'

In [41]:
#tokenizing and padding

tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [42]:
#Neural network

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim),
    tf.keras.layers.GlobalAvgPool1D(),
    tf.keras.layers.Dense(24,activation='relu'),
    tf.keras.layers.Dense(6,activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [43]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 16)          1600000   
_________________________________________________________________
global_average_pooling1d_2 ( (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 24)                408       
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 150       
Total params: 1,600,558
Trainable params: 1,600,558
Non-trainable params: 0
_________________________________________________________________


In [44]:
num_epochs = 3
history = model.fit(training_padded, training_lab, epochs=num_epochs, validation_data=(testing_padded, testing_lab), verbose=2)

Epoch 1/3
3192/3192 - 27s - loss: 0.1227 - accuracy: 0.9353 - val_loss: 0.0808 - val_accuracy: 0.9936
Epoch 2/3
3192/3192 - 27s - loss: 0.0668 - accuracy: 0.9944 - val_loss: 0.0665 - val_accuracy: 0.9936
Epoch 3/3
3192/3192 - 26s - loss: 0.0535 - accuracy: 0.9944 - val_loss: 0.0601 - val_accuracy: 0.9936


In [50]:
#checking model performance

sentence = ['i will kill you, pig', 'i love you']
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model.predict(padded))

[[0.7111781  0.0189842  0.26677603 0.01026183 0.24219564 0.03380141]
 [0.1717326  0.00228816 0.04171696 0.0026091  0.04248479 0.00835407]]


In [51]:
test_data = pd.read_csv('/home/ibab/Desktop/ml_proj/test.csv')
test_data.head(10)

,id,text
0,25f48f649f60423c091b,", 19 May 2006 (UTC)\nThey debate, they don't v..."
1,5c7ac6d7fb400bbadfc7,"""\n\nI am completely nonplussed at this """"We'v..."
2,d00a363d57952496854f,"""\n\nUnblock request\n\nCategory:User block te..."
3,b082c69afa60b378503d,"Dave 1185 \n\nIf you have a moment, can you he..."
4,1a585118ed7e1f29b38b,WarningPlease stop adding nonsense to Wikipedi...
5,df7309a82847b06b9078,"Deepwater deaths \n\nHi, i noticed you were pa..."
6,15c5221a3e2fcf1e0206,"""\n\nActually there is no reason to replace th..."
7,a45988e5c44f83b74d6d,"""I've started a background section here, mostl..."
8,69a53955f44fb1e037b6,Welcome to the target list of operation wikira...
9,2ef1e65d78e5bb41954e,IS THIS A JOKE? \n\nThis article is so biased...


In [79]:
sentence = test_data['text']
ids = test_data['id']
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
predictions = model.predict(padded)
print(ids,predictions)

0        25f48f649f60423c091b
1        5c7ac6d7fb400bbadfc7
2        d00a363d57952496854f
3        b082c69afa60b378503d
4        1a585118ed7e1f29b38b
                 ...         
31910    23e94a16ea98b5bf067c
31911    0112c347e9f7d95f292a
31912    bdcca21bbd42afa26fad
31913    488af1c2ada63cd040f1
31914    570cc22ef59108ba8e0a
Name: id, Length: 31915, dtype: object [[7.4512959e-03 1.5792251e-04 2.7742684e-03 5.0848722e-04 3.4319460e-03
  1.4958978e-03]
 [7.6726923e-05 1.0054716e-08 5.4798948e-06 4.3956183e-07 5.0695144e-06
  3.3491729e-06]
 [5.6339502e-03 1.3002753e-04 2.1618009e-03 4.6977401e-04 2.6930869e-03
  1.3136566e-03]
 ...
 [6.8016627e-05 1.1322316e-09 2.6583225e-06 7.3669888e-08 2.0233156e-06
  9.2508765e-07]
 [5.5246055e-03 5.2125069e-06 8.5994601e-04 3.1952924e-05 8.2764030e-04
  2.0632148e-04]
 [1.6673505e-03 1.6517970e-07 1.3419986e-04 2.0138480e-06 1.1505033e-04
  2.2573642e-05]]


In [75]:
sentence = test_data.loc[888,'text']
sequences = tokenizer.texts_to_sequences([sentence])
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model.predict(padded))

[[9.1109658e-05 7.5445650e-09 5.7825596e-06 3.2464837e-07 5.1511452e-06
  2.8292325e-06]]


In [76]:
sentence

'"\n\nThere is indeed an anchor in the table using the template . Navigation works for me using Internet Explorer 7: clicking on Experientia docet will open List of Latin phrases (C-E) and place the table such that ""experientia docet"" is positioned at the top of the browser\'s window. However, it didnt\'t work here with Firefox 3.0.4; turns out that FF needs exactly agreeing use of case (experientia vs Experientia) when specifying an anchor as target. I have now modified the REDIRECT accordingly.   "'

In [120]:
out_test = pd.DataFrame(columns=['id','harsh','extremely_harsh','vulgar','threatening','disrespect','targeted_hate'])
out_test['id'] = ids
out_test['harsh'] = predictions[:,0]
out_test['extremely_harsh'] = predictions[:,1]
out_test['vulgar'] = predictions[:,2]
out_test['threatening'] = predictions[:,3]
out_test['disrespect'] = predictions[:,4]
out_test['targeted_hate'] = predictions[:,5]

print(out_test)
out_test.to_csv('results.csv',index=False)

                         id     harsh  extremely_harsh    vulgar  \
0      25f48f649f60423c091b  0.007451     1.579225e-04  0.002774   
1      5c7ac6d7fb400bbadfc7  0.000077     1.005472e-08  0.000005   
2      d00a363d57952496854f  0.005634     1.300275e-04  0.002162   
3      b082c69afa60b378503d  0.003272     1.163294e-05  0.000742   
4      1a585118ed7e1f29b38b  0.006289     6.731331e-05  0.001965   
...                     ...       ...              ...       ...   
31910  23e94a16ea98b5bf067c  0.001523     3.673845e-06  0.000304   
31911  0112c347e9f7d95f292a  0.218589     2.848059e-03  0.052875   
31912  bdcca21bbd42afa26fad  0.000068     1.132232e-09  0.000003   
31913  488af1c2ada63cd040f1  0.005525     5.212507e-06  0.000860   
31914  570cc22ef59108ba8e0a  0.001667     1.651797e-07  0.000134   

        threatening  disrespect  targeted_hate  
0      5.084872e-04    0.003432   1.495898e-03  
1      4.395618e-07    0.000005   3.349173e-06  
2      4.697740e-04    0.002693   1.

In [121]:
# import numpy as np
# cols = np.vsplit(predictions, 3)
# print(cols)
!pwd

/home/ibab/Desktop/ml_proj
